In [11]:
# prep

library(tidyverse)
ess18 <- read_csv("https://github.com/CALDISS-AAU/workshop_R-intro/raw/master/data/ESS2018DK_subset.csv")

Rows: 1572 Columns: 16

-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (6): vote, prtvtddk, lvpntyr, tygrtr, gndr, edlvddk
dbl (10): idno, netustm, ppltrst, yrbrn, eduyrs, wkhct, wkhtot, grspnum, frl...


i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [12]:
head(ess18)

idno,netustm,ppltrst,vote,prtvtddk,lvpntyr,tygrtr,gndr,yrbrn,edlvddk,eduyrs,wkhct,wkhtot,grspnum,frlgrsp,inwtm
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
110,180,8,Yes,Socialdemokratiet - The Social democrats,1968,Never too young,Male,1949,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",9,28,28,NA,NA,119
705,60,5,Yes,Det Konservative Folkeparti - Conservative People's Party,1976,67,Male,1958,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",22,37,45,NA,NA,55
1327,240,5,NA,NA,"Still in parental home, never left 2 months",NA,Male,2000,Folkeskole 9.-10. klasse,11,37,37,NA,NA,37
3760,300,7,Not eligible to vote,NA,"Still in parental home, never left 2 months",40,Male,2002,Folkeskole 9.-10. klasse,9,2,2,200,NA,43
4658,90,8,Yes,NA,1974,50,Female,1956,"Kort videregående uddannelse af op til 2-3 års varighed, F.eks. Erhvervsakadem",4,30,30,NA,NA,62
5816,90,7,Yes,SF Socialistisk Folkeparti - Socialist People's Party,1994,60,Male,1974,"Mellemlang videregående uddannelse af 3-4 års varighed. Professionsbachelorer,",35,37,37,37000,35000,61


## Base stats functions

R has a wide range of statistical functions built in (part of base package `stats`).

### Correlation coefficients

Correlation coefficients between two columns can be calculated with `cor()`.

`cor()` contains the arguement "use" specifying what observations to use. By default it will include all observations - including missing. The option "complete.obs" specifies to only calculate for complete observations.

In [16]:
cor(ess18$eduyrs, ess18$netustm, use = "complete.obs")

[1] 0.05662359

Can also be done as a correlation test with `cor.test()`:

In [31]:
cor.test(ess18$eduyrs, ess18$netustm, use = "complete.obs")


	Pearson's product-moment correlation

data:  ess18$eduyrs and ess18$netustm
t = 2.1091, df = 1383, p-value = 0.03511
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.003961935 0.108972044
sample estimates:
       cor 
0.05662359 


### (Student's) t-test

`t.test()` performs both paired and unpaiared t-tests (left-tail, right-tail, two-tail). 

The main arguements are the two columns to be compared (x and y). If only x is specified, it will test whether the mean of the columns is different from zero.

"alternative" specifies the alternative hypothesis. It defaults to a two-sided t-test (`"two-sided"`). Other options are `"less"` for left-tail and `"greater"` for right-tail.

"paired" specifies whether it is a one sample or two sample t-test. It defaults to `FALSE` (unpaired).

Below is a right-tail t-test testing whether people work more than they are contracted to work (paired):

In [37]:
t.test(ess18$wkhtot, ess18$wkhct, alternative = "greater", paired = TRUE)


	Paired t-test

data:  ess18$wkhtot and ess18$wkhct
t = 14.953, df = 1440, p-value < 2.2e-16
alternative hypothesis: true difference in means is greater than 0
95 percent confidence interval:
 2.449922      Inf
sample estimates:
mean of the differences 
               2.752949 


#### T-test between two groups

As an alternative to specifying two columns to compare, you can specify the hypothesis test as a formula.

Specifying hypothesis via formulas is a general feature in R and is used for various statistical modelling. 

Formulas in `t.test()` are specified as: `left-side ~ right-side`. To test differences in means between groups, right-side should be a factor with two levels indicating the groups.

In the standard `t.test()` function, the formula is specified first and then the data. The test belows tests differences in mean work hours between genders:

In [42]:
t.test(wkhtot ~ gndr, ess18)


	Welch Two Sample t-test

data:  wkhtot by gndr
t = -7.2195, df = 1499.2, p-value = 8.26e-13
alternative hypothesis: true difference in means between group Female and group Male is not equal to 0
95 percent confidence interval:
 -6.095408 -3.490832
sample estimates:
mean in group Female   mean in group Male 
            33.56140             38.35452 


https://campus.datacamp.com/courses/hypothesis-testing-in-r/

- korrelation med cor()
- calculation z-scores and p-values
- confidence intervals
- chisquare med chisq.test (https://www.tutorialspoint.com/r/r_chi_square_tests.htm)
- ttest

https://infer.netlify.app/reference/index.html

https://rdrr.io/r/stats/prop.test.html

In [ ]:
late_prop_hyp <- 0.06

# Calculate the standard error
std_error <- late_shipments_boot_distn %>% 
  summarize(sd_late_prop = sd(late_prop)) %>% 
  pull(sd_late_prop)

# Find z-score of late_prop_samp
z_score <- (late_prop_samp - late_prop_hyp) / std_error

# See the results
z_score

In [ ]:
## Right tail

# Calculate the z-score of late_prop_samp
z_score <- (late_prop_samp - late_prop_hyp) / std_error

# Calculate the p-value
p_value <- pnorm(z_score, lower.tail = FALSE)
                 
# See the result
p_value    

In [ ]:
# Calculate 95% confidence interval using quantile method
conf_int_quantile <- late_shipments_boot_distn %>%
  summarize(
    lower = quantile(prop_late_shipments, 0.025),
    upper = quantile(prop_late_shipments, 0.975)
  )

# See the result
conf_int_quantile

In [ ]:
## Two sample mean test statistic by hand
# Calculate the numerator of the test statistic
numerator <- xbar_no - xbar_yes # means for the two groups
 
# Calculate the denominator of the test statistic
denominator <- sqrt(s_no ^ 2 / n_no + s_yes ^ 2 / n_yes) # s for sd n for sample size

# Calculate the test statistic
t_stat <- numerator / denominator

# See the result
t_stat

# Calculate the degrees of freedom
degrees_of_freedom <- n_no + n_yes - 2 # n - number of observations

# Calculate the p-value from the test stat
p_value <- pt(t_stat, df = degrees_of_freedom)

# See the result
p_value

In [ ]:
# Conduct a t-test on diff
test_results <- t.test(sample_dem_data$diff, alternative = "greater") # greater - right tail, lesser - left tail. diff-column differences in between two measures

# See the results
test_results

In [ ]:
# Conduct a paired t-test on dem_percent_12 and dem_percent_16
test_results <- t.test(
  x = sample_dem_data$dem_percent_12,
  y = sample_dem_data$dem_percent_16,
  paired = TRUE,
  alternative = "greater"
)

# See the results
test_results

In [ ]:
# From previous step
mdl_pack_price_vs_shipment_mode <- lm(pack_price ~ shipment_mode, data = late_shipments)

# Perform ANOVA on the regression model
anova(mdl_pack_price_vs_shipment_mode)

In [ ]:
# proportion tests

# Hypothesize that the proportion of late shipments is 6%
p_0 <- 0.06

# Calculate the sample proportion of late shipments
p_hat <- late_shipments %>%
  summarize(prop_late = mean(late == "Yes")) %>%
  pull(prop_late)

# Calculate the sample size
n <- nrow(late_shipments)

# From previous step
p_0 <- 0.06
p_hat <- late_shipments %>%
  summarize(prop_late = mean(late == "Yes")) %>%
  pull(prop_late)
n <- nrow(late_shipments)

# Calculate the numerator of the test statistic
numerator <- p_hat - p_0

# Calculate the denominator of the test statistic
denominator <- sqrt(p_0 * (1 - p_0) / n)

# Calculate the test statistic
z_score <- numerator / denominator

# See the result
z_score

# From previous step
p_0 <- 0.06
p_hat <- late_shipments %>%
  summarize(prop_late = mean(late == "Yes")) %>%
  pull(prop_late)
n <- nrow(late_shipments)
numerator <- p_hat - p_0
denominator <- sqrt(p_0 * (1 - p_0) / n)
z_score <- numerator / denominator

# Calculate the p-value from the z-score
p_value <- pnorm(z_score, lower.tail = FALSE)

# See the result
p_value

In [ ]:
# Perform a chi-square test of independence on
# freight_cost_group and vendor_inco_term
test_results <- late_shipments %>% 
  chisq_test(freight_cost_group ~ vendor_inco_term)

# See the results
test_results

In [ ]:
## simulation based hypothesis testing - proportions

# From previous steps
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )
obs_stat <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# Visualize the null dist'n, adding a vertical line at
# the observed statistic
visualize(null_distn) +
  geom_vline(aes(xintercept = stat), data = obs_stat)

# Get the p-value
p_value <- get_p_value(
  null_distn, obs_stat, 
  direction = "greater"
)

# See the result
p_value

In [ ]:
## simulation based hypothesis testing - means

# From previous steps
null_distn <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 1000, type = "permute") %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))
obs_stat <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))

# Get the p-value
p_value <- get_p_value(
  null_distn, obs_stat,
  direction = "less"
)

# See the result
p_value